# Hyperoptimization with keras-tuner

## 1. Dataset

Example for dataset ESOL from Moleculenet.

In [1]:
import numpy as np
import keras as ks

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
# dataset.set_attributes()
dataset.map_list(method="set_edge_indices_reverse");
dataset.map_list(method="count_nodes_and_edges");
dataset.map_list(**{"method": "count_nodes_and_edges", "total_edges": "total_reverse"});

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL:Read molecules from mol-file.
INFO:kgcnn.data.ESOL: ... process molecules 0 from 1128
INFO:kgcnn.molecule.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.molecule.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2]
INFO:kgcnn.molecule.encoder:OneHotEncoder TotalDegree found [2, 4, 1, 3]
INFO:kgcnn.molecule.encoder:OneHotEncoder TotalNumHs found [1, 2, 0, 3, 4]
INFO:kgcnn.molecule.encoder:OneHotEncoder CIPCode found [None, 'S', 'R'

## 2. Model

Pick DMPNN as example model.

In [3]:
from kgcnn.literature.DMPNN import make_model

In [4]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": (None, 41), "name": "node_attributes", "dtype": "float32"},
        {"shape": (None, 11), "name": "edge_attributes", "dtype": "float32"},
        {"shape": (None, 2), "name": "edge_indices", "dtype": "int64"},
        {"shape": (None, 1), "name": "edge_indices_reverse", "dtype": "int64"},
        {"shape": (), "name": "total_nodes", "dtype": "int64"},
        {"shape": (), "name": "total_edges", "dtype": "int64"},
        {"shape": (), "name": "total_reverse", "dtype": "int64"}
    ],
    "cast_disjoint_kwargs": {},
    "input_node_embedding": {"input_dim": 95, "output_dim": 64},
    "input_edge_embedding": {"input_dim": 5, "output_dim": 64},
    "input_graph_embedding": {"input_dim": 100, "output_dim": 64},
    "pooling_args": {"pooling_method": "scatter_sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "graph",
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.models.utils:Updated model kwargs: '{'name': 'DMPNN', 'inputs': [{'shape': (None, 41), 'name': 'node_attributes', 'dtype': 'float32'}, {'shape': (None, 11), 'name': 'edge_attributes', 'dtype': 'float32'}, {'shape': (None, 2), 'name': 'edge_indices', 'dtype': 'int64'}, {'shape': (None, 1), 'name': 'edge_indices_reverse', 'dtype': 'int64'}, {'shape': (), 'name': 'total_nodes', 'dtype': 'int64'}, {'shape': (), 'name': 'total_edges', 'dtype': 'int64'}, {'shape': (), 'name': 'total_reverse', 'dtype': 'int64'}], 'input_tensor_type': 'padded', 'cast_disjoint_kwargs': {}, 'input_embedding': None, 'input_node_embedding': {'input_dim': 95, 'output_dim': 64}, 'input_edge_embedding': {'input_dim': 5, 'output_dim': 64}, 'input_graph_embedding': {'input_dim': 100, 'output_dim': 64}, 'pooling_args': {'pooling_method': 'scatter_sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activatio

In [5]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property 'edge_attributes' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices_reverse' is an empty list for graph '934'.


array([934])

In [6]:
labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
labels.shape

(1127, 1)

## 3. Keras Tuner

Optimize hyperparameter with keras tuner.

In [8]:
import warnings
import keras_tuner as kt
from copy import deepcopy
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

ModuleNotFoundError: No module named 'keras_core'

In [ ]:
train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
x_train, y_train = dataset[train_index].tensor(model_config["inputs"]), labels[train_index]
x_valid, y_valid = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]

In [ ]:
# Define model build()
def build_model(hp):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    hyper_trial = deepcopy(model_config)    
    hyper_trial["depth"] = hp.Int('depth', min_value=3, max_value=5, step=1)
    hyper_trial["pooling_args"]["pooling_method"] = hp.Choice("pooling_method", ["scatter_sum", "scatter_mean"])
    units = hp.Int('nn_size', min_value=25, max_value=400, step=25)
    hyper_trial["node_dense"]["units"] = units
    hyper_trial["edge_dense"]["units"] = units
    hyper_trial["edge_initialize"]["units"] = units
    # print(hyper_trial)
    
    model = make_model(**hyper_trial)
    
    # We compile our model
    learning_rate = hp.Choice('lr_start', [1e-3, 5e-4, 1e-4])
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["mean_absolute_error"],
    )

    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_loss',
                     max_epochs=10, factor=3, directory="kt_dmpnn")

In [ ]:
tuner.search(x_train,y_train,
            shuffle=True,
            epochs=100, validation_data=(x_valid, y_valid))

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
    Search complete, best hyperparameters:
    depth = {best_hps.get('depth')}
    nn_size = {best_hps.get('nn_size')}
    ene_wt = {best_hps.get('pooling_method')}
    learn rate = {best_hps.get('lr_start')}
""")